In [ ]:
####
# Creator: Rowan Converse (rowanconverse@unm.edu)
# Date: 2022/08/02
# Purpose: Scratch, testing pieces of the full code to translate raw labels generated by volunteers on the Zooniverse platform into COCO format
# Ref COCO Camera Trap Standard: https://cocodataset.org/#format-data
####

In [6]:
#Loading JSON within CSV
import pandas as pd
import json
import datetime
from collections import OrderedDict 

zooniverse = pd.read_csv('Data/zooniverse_snippet.csv')

In [ ]:
#Benni's example: load first row's annotations
row0_anno = json.loads(data.annotations[0])
print('Row 0: number of annotations = {}'.format(len(row0_anno)))

first_anno = row0_anno[0]['value'][0]    # extract the first of two annotations of the first row, then its properties under 'value' (dict), and again the first entry (that property is a list)
print('First annotation: bbox = {}, {}, {}, {}; label = {}'.format(
    first_anno['x'], first_anno['y'],
    first_anno['width'], first_anno['height'],
    first_anno['tool_label']
))

In [19]:
# Statistics about annotations
row0_anno = json.loads(data.annotations[0])
first_anno = row0_anno[0]['value'][0]
print(first_anno)

#annotations = []
#for i in range(len(data)):
#       for x in data[i]["value"]:
#              annotations.append(x)

#print(len(annotations))

{'x': 575.9765625, 'y': 265.39453125, 'tool': 0, 'frame': 0, 'width': 79.1796875, 'height': 94.40625, 'details': [], 'tool_label': 'Duck'}


TypeError: the JSON object must be str, bytes or bytearray, not Series

In [26]:
#Statistics about images
images = data.subject_ids

print("Number of Images: "+str(len(images.unique())))
#print('Animal images: '+str(len([ann['value'] for ann in anno if ann['value'] != 0])))
#print('Empty images: '+str(len([ann['value'] for ann in anno if ann['value'] == 0])))

Number of Images: 282


In [ ]:
for i in range(len(zooniverse)):
  imglist = list(set(img['External ID'] for img in zooniverse))

#Add unique IDs to each filename
list_ids = [{v: k for k, v in enumerate(
   OrderedDict.fromkeys(imglist), 1)}
      [n] for n in imglist]
img = dict(zip(list_ids, imglist))

In [4]:
#Statistics about labelers
users = data.user_id
print("Number of Labelers: "+str(len(users.unique())))

0      2091290
1      2313201
2      2313184
3      2313201
4      2313202
        ...   
278    2141263
279    2154838
280    2142076
281    2247823
282    2316349
Name: user_id, Length: 283, dtype: int64

In [ ]:
#Number/Type of Categories

In [ ]:
#Empty Images

In [ ]:
#Annotations per Image

In [ ]:
#Annotations per Category

In [ ]:
#Filtered Mean of Counts

#Remember to create final dataset where area = minimum overlap of consensus bbox locations

In [ ]:
#Plurality vote of identifications

#Assign based on overlap between labelers and between the filtered mean set
#Produce refined label set of ids x counts